<a href="https://colab.research.google.com/github/passionforcodez/Python/blob/main/AIChessWithGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio python-chess cairosvg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 96.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=f471485f96519b992db46f5805a2e9244ed3e451be77c3b33cee09b0ce938e71
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [ ]:
# gradio_chess_ai.py
import chess
import chess.svg
import math
import gradio as gr
from html import escape

# --- Simple evaluator (material + small piece-square tables) ---
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

# Small piece-square tables for simple positional bias (white perspective).
# These are tiny and optional — they give the AI slightly smarter play.
PAWN_TABLE = [
     0,  0,  0,  0,   0,  0,  0,  0,
     5, 10, 10,-20, -20, 10, 10,  5,
     5, -5,-10,  0,   0,-10, -5,  5,
     0,  0,  0, 20,  20,  0,  0,  0,
     5,  5, 10, 25,  25, 10,  5,  5,
    10, 10, 20, 30,  30, 20, 10, 10,
    50, 50, 50, 50,  50, 50, 50, 50,
     0,  0,  0,  0,   0,  0,  0,  0
]

KNIGHT_TABLE = [
   -50,-40,-30,-30,-30,-30,-40,-50,
   -40,-20,  0,  0,  0,  0,-20,-40,
   -30,  0, 10, 15, 15, 10,  0,-30,
   -30,  5, 15, 20, 20, 15,  5,-30,
   -30,  0, 15, 20, 20, 15,  0,-30,
   -30,  5, 10, 15, 15, 10,  5,-30,
   -40,-20,  0,  5,  5,  0,-20,-40,
   -50,-40,-30,-30,-30,-30,-40,-50
]

BISHOP_TABLE = [
   -20,-10,-10,-10,-10,-10,-10,-20,
   -10,  0,  0,  0,  0,  0,  0,-10,
   -10,  0,  5, 10, 10,  5,  0,-10,
   -10,  5,  5, 10, 10,  5,  5,-10,
   -10,  0, 10, 10, 10, 10,  0,-10,
   -10, 10, 10, 10, 10, 10, 10,-10,
   -10,  5,  0,  0,  0,  0,  5,-10,
   -20,-10,-10,-10,-10,-10,-10,-20
]

ROOK_TABLE = [
     0,  0,  5, 10, 10,  5,  0,  0,
     0,  0,  5, 10, 10,  5,  0,  0,
     0,  0,  5, 10, 10,  5,  0,  0,
     0,  0,  5, 10, 10,  5,  0,  0,
     0,  0,  5, 10, 10,  5,  0,  0,
     0,  0,  5, 10, 10,  5,  0,  0,
    25, 25, 25, 25, 25, 25, 25, 25,
     0,  0,  5, 10, 10,  5,  0,  0
]

QUEEN_TABLE = [0]*64
KING_TABLE = [0]*64

PSQT = {
    chess.PAWN: PAWN_TABLE,
    chess.KNIGHT: KNIGHT_TABLE,
    chess.BISHOP: BISHOP_TABLE,
    chess.ROOK: ROOK_TABLE,
    chess.QUEEN: QUEEN_TABLE,
    chess.KING: KING_TABLE
}

def evaluate_board(board: chess.Board) -> int:
    """Return evaluation from White's perspective in centipawns."""
    if board.is_checkmate():
        # If checkmate, huge positive if White wins, huge negative if Black wins
        return 100000 if board.turn == chess.BLACK else -100000
    if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return 0

    eval_score = 0
    for piece_type in PIECE_VALUES:
        for square in board.pieces(piece_type, chess.WHITE):
            eval_score += PIECE_VALUES[piece_type]
            eval_score += PSQT.get(piece_type, [0]*64)[square]
        for square in board.pieces(piece_type, chess.BLACK):
            eval_score -= PIECE_VALUES[piece_type]
            # mirror the tables for black
            eval_score -= PSQT.get(piece_type, [0]*64)[chess.square_mirror(square)]

    return eval_score

# --- Negamax with alpha-beta pruning ---
def negamax(board: chess.Board, depth: int, alpha: int, beta: int) -> (int, chess.Move):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board), None

    best_value = -9999999
    best_move = None
    # order moves: prioritize captures (simple move ordering)
    moves = list(board.legal_moves)
    moves.sort(key=lambda m: 0 if board.is_capture(m) else 1)
    for move in moves:
        board.push(move)
        val, _ = negamax(board, depth - 1, -beta, -alpha)
        val = -val
        board.pop()

        if val > best_value:
            best_value = val
            best_move = move
        alpha = max(alpha, val)
        if alpha >= beta:
            break
    return best_value, best_move

def get_ai_move(board: chess.Board, depth: int = 2) -> chess.Move:
    # If few pieces left, boost depth a bit
    non_pawn_material = sum(1 for p in board.piece_map().values() if p.piece_type != chess.PAWN)
    effective_depth = depth + (1 if non_pawn_material < 6 else 0)
    _, move = negamax(board, effective_depth, -10000000, 10000000)
    return move

# --- Gradio helpers ---
def board_to_svg_html(board: chess.Board, last_move: chess.Move = None) -> str:
    """Return an embeddable HTML string containing the SVG board."""
    svg = chess.svg.board(board=board, lastmove=last_move, size=480)
    # Wrapping SVG safely
    return f"<div style='display:flex;justify-content:center'>{svg}</div>"

def game_step(user_move: str, fen: str, ai_depth: int):
    """
    Gradio callback:
     - user_move: string like 'e2e4' or 'Nf3' or 'O-O'
     - fen: current FEN (or empty -> start position)
     - ai_depth: integer (AI search depth)
    Returns: (html_board, new_fen, status_message)
    """
    # initialize board
    board = chess.Board(fen) if fen else chess.Board()
    status = ""

    # If game already over, reset board
    if board.is_game_over():
        status = f"Game over: {board.result()} — {board.outcome().termination.name}"
        return board_to_svg_html(board), board.fen(), status

    # Attempt to parse and apply user's move
    try:
        # Accept either UCI (e2e4) or LAN/SAN (Nf3)
        move = None
        # Try SAN first for user friendliness
        try:
            move = board.parse_san(user_move.strip())
        except Exception:
            # Fall back to UCI
            try:
                move = chess.Move.from_uci(user_move.strip())
            except Exception:
                move = None

        if move is None or move not in board.legal_moves:
            status = "Illegal move or couldn't parse. Enter SAN (e.g. Nf3), UCI (e2e4), or O-O/O-O-O for castling."
            return board_to_svg_html(board), board.fen(), status

        board.push(move)
    except Exception as e:
        status = f"Error applying your move: {escape(str(e))}"
        return board_to_svg_html(board), board.fen(), status

    # Check for game end after user's move
    if board.is_game_over():
        status = f"You played {board.peek().uci()}. Game over after your move: {board.result()} — {board.outcome().termination.name}"
        return board_to_svg_html(board, last_move=board.peek()), board.fen(), status

    # AI move
    ai_move = get_ai_move(board, depth=ai_depth)
    if ai_move is None:
        status = "AI could not find a move (draw?)."
        return board_to_svg_html(board), board.fen(), status

    board.push(ai_move)

    # Prepare status
    status = f"You played {board.peek().uci()} and AI replied {ai_move.uci()}. "
    if board.is_check():
        status += "Check! "
    if board.is_game_over():
        status += f"Game over: {board.result()} — {board.outcome().termination.name}"

    return board_to_svg_html(board, last_move=ai_move), board.fen(), status

def reset_game():
    board = chess.Board()
    return board_to_svg_html(board), board.fen(), "New game. You play White. Enter moves in SAN (e.g., Nf3) or UCI (e2e4)."

# --- Build Gradio interface ---
with gr.Blocks() as demo:
    gr.Markdown("## ♟️ AI Chess (Python + Gradio)\nI play Black and will respond with moves using a small negamax AI. Enter your move in SAN (e.g. `Nf3`) or UCI (e.g. `e2e4`).")
    with gr.Row():
        board_html = gr.HTML(board_to_svg_html(chess.Board()))
    with gr.Row():
        move_in = gr.Textbox(label="Your move (SAN or UCI)", placeholder="e.g. e2e4 or Nf3 or O-O", interactive=True)
        ai_depth = gr.Slider(label="AI depth (higher => stronger/slower)", minimum=1, maximum=4, step=1, value=2)
    # hidden fen state
    fen_state = gr.Textbox(value=chess.Board().fen(), visible=False)
    status_out = gr.Text(label="Status")
    # Buttons
    play_btn = gr.Button("Play Move")
    reset_btn = gr.Button("Reset")

    play_btn.click(fn=game_step, inputs=[move_in, fen_state, ai_depth], outputs=[board_html, fen_state, status_out])
    reset_btn.click(fn=reset_game, inputs=[], outputs=[board_html, fen_state, status_out])

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f0f15e2f28ecb8556.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import chess
import chess.svg
import io
from cairosvg import svg2png
from PIL import Image
import random

# Initialize global board
board = chess.Board()

def board_to_image(board):
    """Convert the current chess board to a PNG image."""
    svg_data = chess.svg.board(board=board).encode("utf-8")
    png_data = svg2png(bytestring=svg_data)
    return Image.open(io.BytesIO(png_data))

def ai_move():
    """AI makes a random valid move."""
    moves = list(board.legal_moves)
    if moves:
        move = random.choice(moves)
        board.push(move)

def play_move(move_uci):
    """Handle user's move and generate new board state."""
    global board

    if board.is_game_over():
        return board_to_image(board), "Game over! Reset to play again."

    try:
        move = chess.Move.from_uci(move_uci)
        if move in board.legal_moves:
            board.push(move)
        else:
            return board_to_image(board), "❌ Invalid move! Try again."
    except Exception:
        return board_to_image(board), "⚠️ Invalid input! Use UCI format (e.g., e2e4)."

    # AI move
    if not board.is_game_over():
        ai_move()

    # Status message
    status = ""
    if board.is_checkmate():
        status = "♚ Checkmate!"
    elif board.is_stalemate():
        status = "🤝 Stalemate!"
    elif board.is_check():
        status = "⚠️ Check!"
    else:
        status = "Your move."

    return board_to_image(board), status

def reset_board():
    """Reset the game."""
    global board
    board.reset()
    return board_to_image(board), "New game started. You play white!"

# Initial setup
init_image = board_to_image(board)
with gr.Blocks() as demo:
    gr.Markdown("## ♟️ AI Chess (SVG Version - Works in Colab)")
    gr.Markdown("Enter your move in UCI notation (e.g., `e2e4`). The AI will respond automatically.")

    with gr.Row():
        board_image = gr.Image(value=init_image, label="Chess Board", interactive=False)
        move_box = gr.Textbox(label="Your Move (e.g., e2e4)")

    with gr.Row():
        play_btn = gr.Button("Play Move")
        reset_btn = gr.Button("Reset Game")

    status_text = gr.Textbox(value="Your move.", label="Status", interactive=False)

    play_btn.click(play_move, inputs=move_box, outputs=[board_image, status_text])
    reset_btn.click(reset_board, inputs=None, outputs=[board_image, status_text])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f664aa1b33767499b4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import chess
import chess.svg
import io
from cairosvg import svg2png
from PIL import Image
import random

board = chess.Board()
selected_square = None

def board_to_image():
    """Convert current board to image."""
    svg_data = chess.svg.board(board=board).encode("utf-8")
    png_data = svg2png(bytestring=svg_data)
    return Image.open(io.BytesIO(png_data))

def handle_click(square):
    """Handle clicks — select or move piece."""
    global selected_square, board

    # If game is over
    if board.is_game_over():
        return board_to_image(), "Game over! Reset to play again."

    # If no square selected yet
    if selected_square is None:
        piece = board.piece_at(chess.parse_square(square))
        if piece and piece.color == chess.WHITE:
            selected_square = square
            return board_to_image(), f"Selected {square} — choose where to move."
        else:
            return board_to_image(), "⚠️ Select a valid white piece."

    # If already selected, try to move
    move = chess.Move.from_uci(selected_square + square)
    if move in board.legal_moves:
        board.push(move)
        selected_square = None

        # AI move
        if not board.is_game_over():
            ai_move = random.choice(list(board.legal_moves))
            board.push(ai_move)

        # Status
        if board.is_checkmate():
            return board_to_image(), "♚ Checkmate!"
        elif board.is_stalemate():
            return board_to_image(), "🤝 Stalemate!"
        elif board.is_check():
            return board_to_image(), "⚠️ Check!"
        else:
            return board_to_image(), "Your move."

    else:
        selected_square = None
        return board_to_image(), "❌ Invalid move. Try again."

def reset_game():
    """Reset board and clear state."""
    global board, selected_square
    board = chess.Board()
    selected_square = None
    return board_to_image(), "New game started. You play white!"

# --- Gradio UI ---
squares = [f"{f}{r}" for r in range(8, 0, -1) for f in "abcdefgh"]

with gr.Blocks() as demo:
    gr.Markdown("## ♟️ Click-Based AI Chess (No JS, Works in Colab)")
    gr.Markdown("Click a piece, then click the square you want to move to.")

    board_img = gr.Image(value=board_to_image(), label="Board", interactive=False)
    status = gr.Textbox(value="Your move.", label="Status", interactive=False)

    with gr.Row():
        for rank in range(8):
            with gr.Row():
                for file in range(8):
                    sq = f"{'abcdefgh'[file]}{8 - rank}"
                    btn = gr.Button(sq, scale=1)
                    btn.click(handle_click, inputs=gr.State(sq), outputs=[board_img, status])

    gr.Button("♻️ Reset Game").click(reset_game, outputs=[board_img, status])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://872cba6f29804f9392.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
